In [2]:
#Загрузка данных
import os
from six.moves import urllib
DOWNLOAD_LINK = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
WINE_LINK = os.path.join("data","wine")
def download_wine(dl = DOWNLOAD_LINK, wp = WINE_LINK):
    if not os.path.isdir(wp):
        os.makedirs(wp)
    WINE_LINK = wp
    d_path = os.path.join(wp,"wine.data")
    urllib.request.urlretrieve(dl,d_path)

In [3]:
import numpy as np
from scipy.special import expit
class neural_network:
    def __init__ (self, input_nodes, hiden_nodes, output_nodes, learning_rate):
        
        self.hl = (np.random.rand(hiden_nodes, input_nodes) - 0.5)
        self.ol = (np.random.rand(output_nodes, hiden_nodes) -0.5)
        
        self.lr = learning_rate
        self.activation_function = lambda x : expit(x)
        
    def query (self, input_list):
        input_l = np.array(input_list, ndmin=2).T
        
        hiden_input = np.dot(self.hl, input_l)
        hiden_output = self.activation_function(hiden_input) 
        
        final_input = np.dot(self.ol, hiden_output)
        final_output = self.activation_function(final_input)
        return final_output
    def train (self, input_list, target_list):
        input_l = np.array(input_list, ndmin=2).T
        resalt = np.array(target_list, ndmin = 2).T
        
        hiden_input = np.dot(self.hl, input_l)
        hiden_output = self.activation_function(hiden_input) 
        
        final_input = np.dot(self.ol, hiden_output)
        final_output = self.activation_function(final_input)
        
        o_error = resalt - final_output
        h_error = np.dot(self.ol.T,o_error)
        
        self.ol += self.lr*np.dot((o_error*final_output * (1-final_output)),hiden_output.T)
        self.hl += self.lr*np.dot((h_error*hiden_output * (1-hiden_output)),input_l.T)

In [148]:
#Загрузка и нормировка
download_wine()
with open(os.path.join(WINE_LINK,"wine.data")) as file:
        data_list = np.array([list(map(float,line.split(","))) for line in file])
for i in range (1,data_list.shape[1]):
    temp = data_list[:,i]
    temp/= temp.max()+1

#Разделение данных
np.random.seed(42)
testing_index = set(np.random.randint(0,data_list.shape[0], data_list.shape[0]//5))
training_list,testing_list = [],[]
np.random.shuffle(data_list)

for i in range(data_list.shape[0]):
    if  i in testing_index:
        testing_list.append(data_list[i])
    else:
        training_list.append(data_list[i])
nn1_wine = neural_network(13,300,3,0.6)

In [149]:
#Обучение
epochs =9
for j in range(epochs):
    for i in training_list:
        t_example = i[1::]
        t = np.zeros((1,3))
        t[0,int(i[0])-1]+=0.99
        nn1_wine.train(t_example,t)
    

In [150]:
#Тест
res=0
for i in testing_list:
    t_example = i[1::]
    label = np.argmax(nn1_wine.query(t_example))+1
    if label==i[0]:
        res+=1
print(res/len(testing_list))    

0.967741935483871
